# Notebook for Computer vision final presentation

This notebook include the code to run the different modules requested for this project.
Instance classification is done online while the other modules are executed after the sequence is fully acquired.

In [13]:
# imports
import time
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import math
import imutils
import torch
import re

from imutils import paths
from tkinter import Image
from PIL import ImageOps, Image
import pickle

## Program parameters

In [14]:
"""
panorama = fuse_panorama_weighted(panorama, warped_im, warped_mask, known_mask)

INPUT : 
- panorama : the current average panorama
- warped_im : the new image to add to the panorama
- warped_mask : mask for warped_im
- known_mask : mask of panorama
OUTPUT :
- "res" : panorama with the new warped_im. For the parts of the panorama that were not known
          the corresponding part of image is simply added.
          The other are blended using a weight value
"""
def fuse_panorama_weighted(panorama, warped_im, warped_mask, known_mask, weight):
    inverted_mask = cv2.bitwise_not(warped_mask) # 0 where image is 1 everywhere else

    panorama_new = cv2.bitwise_and(inverted_mask,panorama) #eliminate the new part of the image
    panorama_new = cv2.bitwise_or(panorama_new,warped_im)      #copy new part into panorama


    # KNOWN MASK : the parts of the image that have never been explored can be directly added onto the image without weighting
    new_part_mask = cv2.bitwise_and(warped_mask, cv2.bitwise_xor(warped_mask, known_mask))
    inverted_new_part_mask = cv2.bitwise_not(new_part_mask)
    panorama = cv2.bitwise_and(inverted_new_part_mask,panorama)
    warped_im_to_insert = cv2.bitwise_and(new_part_mask, warped_im)
    panorama = cv2.bitwise_or(panorama, warped_im_to_insert)

    #now do the weighting between panorama_new and panorama
    panorama_new = np.float32(panorama_new)
    panorama = np.float32(panorama)
    cv2.accumulateWeighted(panorama_new, panorama, weight, warped_mask)
    res = cv2.convertScaleAbs(panorama)

    return res 

In [15]:
# set program parameters 

# camera parameters
K = np.array([[ 949.08854289, 0., 666.31415728],
              [ 0., 951.10897066,  369.93327023],
              [ 0., 0.,        1.]])

#step size between processed frames
step = 5

#first processed image
first_im = 20


#If bounding boxes are already computed
presegmented = True
test_seq = True

## helper functions

In [16]:
def masked_image_deletion(im,corners1 = list, corners2 = list):
    print(type(corners1))
    mask = np.full((im.shape[0],im.shape[1]),0, dtype="uint8") #white mask
    for i in range(len(corners1)):
        cv2.rectangle(mask, (int(corners1[i][0]),int(corners1[i][1])), (int(corners2[i][0]),int(corners2[i][1])), 255, -1) # black rectangles
    return cv2.bitwise_and(im,mask)

def fuse_panorama_bitwise(panorama,warped_im,warped_mask):
    inverted_mask = cv2.bitwise_not(warped_mask) # 0 where image is 1 everywhere else
    panorama = cv2.bitwise_and(inverted_mask,panorama) #eliminate the new part of the image
    panorama = cv2.bitwise_or(panorama,warped_im)      #copy new part into panorama
    return panorama


## Image Acquisition + Object Detection

In [17]:
img_array = []
filenames = []
n_images = 600 #number of images to load

if not presegmented :
    model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)
    model.classes = [0, 37]
    model.conf = 0.20
    model.iou = 0.60

results_df = []
foreground_masks = []
for filename in glob.glob('../test_sequence/test_sequence/*.png'):
    
    filenames.append(filename)

filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

tic = time.perf_counter()

classes = []
boxes = []

if presegmented:

    classes = pickle.load( open( "classes.pkl", "rb") )
    boxes   = pickle.load( open( "boxes.pkl", "rb") )
for i, filename in enumerate(filenames):
    ## get image
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape[:2]
    size = (width,height)
    img_array.append(img)

    if not presegmented : 
        #predict and save boundng boxes in a dataframe
        result = model(img, size=640)
        results_df.append(result.pandas().xyxy[0])

    if i %100 == 0:
        #cv2.imshow('res', result.render()[0])
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        print(i)
    

    #make and save background masks. 255 = foreground, 0 = background
    corners_min = []
    corners_max = []
    if not presegmented :

        for index, row in results_df[i].iterrows():
            corners_min.append((row['xmin'], row['ymin']))
            corners_max.append((row['xmax'], row['ymax']))
    else:
        for j, rect in enumerate(boxes[i]):

            corners_min.append((rect[0],rect[1]))
            corners_max.append((rect[2], rect[3]))            

    mask = np.full((size[1],size[0]),0, dtype="uint8") #black mask
    for j in range(len(corners_min)):

        cv2.rectangle(mask, (int(corners_min[j][0]),int(corners_min[j][1])), (int(corners_max[j][0]),int(corners_max[j][1])), 255, -1) # white rectangles
    
    foreground_masks.append(mask)
toc = time.perf_counter()
print(f"Boxed the sequence in {toc - tic:0.4f} seconds")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
Boxed the sequence in 26.0164 seconds


In [18]:
cv2.imshow("foreground", foreground_masks[500])
cv2.waitKey(0)
cv2.destroyAllWindows()

out = cv2.VideoWriter('masks_test.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, (1280,720), 0)
for i, img in enumerate(foreground_masks):
    out.write(img)

out.release()

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to tar

## Rotation estimation


### Auxiliary function for rotation estimation

In [19]:
# Code taken from: https://learnopencv2.com/rotation-matrix-to-euler-angles/ 
# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

# Calculates rotation matrix to euler angles.
def rotationMatrixToEulerAngles(R) :

    assert(isRotationMatrix(R))
 
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
 
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])

# Calculate angle from rotation matrix.
def rotationMatrixToAngle(R):
    return

#compute Y rotation matrix from euler angle
def getYRotationMatrixFromAngle(y_angle):

    return np.array([[np.cos(y_angle),0,np.sin(y_angle)],
                     [0,              1,              0],
                     [-np.sin(y_angle),0,np.cos(y_angle)]])


def findBestMatches(img1, img2, background_mask1,background_mask2):

    # Detect points of interests in the two images.
    sift = cv2.SIFT.create(nfeatures = 2000)
    kp1, des1 = sift.detectAndCompute(img1,background_mask1)
    kp2, des2 = sift.detectAndCompute(img2,background_mask2)
    
    # Match points between the two images.
    bf = cv2.DescriptorMatcher.create(2)
    matches = bf.match(des1,des2)
    
    '''
    # Find the best matches.
    best_matches = []
    for matche in matches:
        if matche.distance < 25:
            best_matches.append(matche)
    
    # Keep only the 10 best matches.
    best_matches = sorted(best_matches, key = lambda x:x.distance)[0:10] # quid if less than 10 that are < 25 ?
    '''
    best_matches = sorted(matches, key=lambda x: x.distance)
    
    # Get the coordinates of the best matching points.
    points1 = []
    points2 = []
    for matche in best_matches:
        points1.append(kp1[matche.queryIdx].pt)
        points2.append(kp2[matche.trainIdx].pt)
    
    points1 = np.array(points1)
    points2 = np.array(points2)
    
    return points1, points2

def getAnglesBetweenImages(img1, img2,background_mask1,background_mask2):

    K = np.array([[ 949.08854289, 0, 666.31415728],
                  [ 0, 951.10897066,  369.93327023],
                  [ 0, 0, 1]])
    
    #K = np.eye(3)

    points1, points2 = findBestMatches(img1, img2,background_mask1,background_mask2)

    E, mask = cv2.findEssentialMat(points1, points2, method=cv2.RANSAC, cameraMatrix=K, prob = 0.99999, threshold = 1.0) # + give camera intrinsic parameters
    
    retval, R, t, mask = cv2.recoverPose(E,points1, points2, cameraMatrix=K) # mask to considere only best matching point ? (inliner)

    x, y, z = rotationMatrixToEulerAngles(R)
    
    # Reject big angles (small angles hypothesis).
    if np.abs(y) < 0.1:
        return y
    else:
        return np.nan

def getRotMatrixBetweenImages(img1, img2,background_mask1, background_mask2):

    K = np.array([[ 949.08854289, 0, 666.31415728],
                  [ 0, 951.10897066,  369.93327023],
                  [ 0, 0, 1]])

    points1, points2 = findBestMatches(img1, img2,background_mask1, background_mask2)

    E, mask = cv2.findEssentialMat(points1, points2, method=cv2.RANSAC, cameraMatrix=K, prob = 0.999999, threshold = 1.0) # + give camera intrinsic parameters
    
    retval, R, t, mask = cv2.recoverPose(E,points1, points2, cameraMatrix=K) # mask to considere only best matching point ? (inliner)

    x, y, z = rotationMatrixToEulerAngles(R)
    
    # Reject big angles (small angles hypothesis).
    if np.abs(y) < 0.1:
        return y , R
    else:
        return np.nan, R

def writeAngle(img,thing):
  position = (10,50)
  cv2.putText(
     img, #numpy array on which text is written
     str(thing), #text
     position, #position at which writing has to start
     cv2.FONT_HERSHEY_SIMPLEX, #font family
     1, #font size
     (209, 80, 0, 255), 
     3) #font stroke
  return img

### Rotation estimation for each image

In [20]:
rotation_matrices = []
prev_R = np.eye(3)
tot_R = np.eye(3)

angles = []
previous_angle = 0.0
angle = 0.0
prev_update = 0


tic = time.perf_counter()
for i in range(first_im, len(img_array) - step, step):
    if i %100 == 0:
        print(i)
    #compute rotation matrix
    update, curr_R = getRotMatrixBetweenImages(img_array[i], img_array[i+step],None,None) # previous image

    # Use the previous update if the current one cannot be computed.
    if np.isnan(update):
        print("Nan value...\n", (angle+prev_update)* 180/np.pi)
        update = prev_update 
        curr_R = prev_R
    else:
        prev_update = update
        prev_R = curr_R

    # total rotation update
    tot_R = np.dot(tot_R,curr_R.transpose())
    angle = angle + update

    # save rotation matrix
    angles.append(angle)
    rotation_matrices.append(tot_R)
toc = time.perf_counter()
print(f"Estimated rotation matricies for sequence in {toc - tic:0.4f} seconds")

100
200
300
400
500
600
700
800
900
1000
1100
1200
Nan value...
 37.36689597593273
1300
1400
Estimated rotation matricies for sequence in 99.8814 seconds


## Movement detection

In [21]:
def masked_image_deletion(im,foreground_mask):
    
    return cv2.bitwise_and(im,foreground_mask)


In [22]:
# load ground truth masks

def get_numbers_from_filename(filename):
    return re.search(r'\d+', filename).group(0)

# contains ground truth masks
ground_truth_masks = []
ground_truth_masks_indexes = []
ground_truth_path = "../sequences_2/group_3/masks/*.png"

filenames_ground_truth = []
results_df = []
for filename in glob.glob(ground_truth_path):
    filenames_ground_truth.append(filename)

image_indexes = []
for i, filename in enumerate(filenames_ground_truth):
    r = re.compile("[0-9]{4}")
    index = r.search(filename).group(0)
    ground_truth_masks_indexes.append(int(index))

    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    ground_truth_masks.append(img)

print(ground_truth_masks_indexes)




[168, 366, 716, 1336, 227, 772, 436, 826, 1020, 487, 913, 618, 552, 1470, 661, 138, 263, 1435, 1052, 513, 1076, 918, 505, 1430, 964, 1132, 660, 1287, 1082, 291, 187, 974, 485, 586, 1423, 1124, 1265, 393, 722, 862, 1234, 535, 196, 1478, 1204, 1007, 849, 1250, 250, 646]


In [23]:


"""
seg_masks, seg_masks_GT, seg_masks_compare = getForeGroundMasks(img_array, img_indexes, ground_truth_masks, ground_truth_masks_indexes, bounding_boxes)

INPUT :
- img_array : array of images 
- img_indexes : their indexes 
- ground_truth_masks : array of ground truth segmentation masks
- ground_truth_masks_indexes : their indexes
- bounding_boxes : array of masks of bounding boxes

OUTPUT
- array of panorama segmentation masks
- array of warped user made segmentation masks
"""
def getForegroundMasks(img_array, img_indexes, ground_truth_masks, ground_truth_masks_indexes, bounding_boxes):

    # camera essential matrix
    K = np.array([[ 949.08854289, 0, 666.31415728],
            [ 0, 951.10897066,  369.93327023],
            [ 0, 0, 1]])

    # total rotation matrix
    tot_R = np.eye(3) 
    
    angle = 0
    angles = []
    rotation_matricies = []
    prev_update = 0
    epsylon = 1
    
    
    orientation = True
    rotation = 0.0
    tmp_index = 0

    while(((rotation * 180/np.pi < epsylon and rotation * 180/np.pi > -epsylon) or np.isnan(rotation)) and tmp_index < len(img_array) - 1):
        # first invert bounding boxes
        
        bg1 = cv2.bitwise_not(bounding_boxes[tmp_index])
        bg2 = cv2.bitwise_not(bounding_boxes[tmp_index+1])
        rotation, _ = getRotMatrixBetweenImages(img_array[tmp_index], img_array[tmp_index+1],bg1, bg2)
        if rotation > 0.0:
            orientation = True
        if rotation < 0.0:
            orientation = False
        tmp_index += 1

    # Images are rotated 180° if they are upside down
    if orientation:
        for i in range(0,len(img_array)):
            img_array[i] = cv2.rotate(img_array[i], cv2.ROTATE_180)

    # Initialize cylindrical projection warper
    warper = cv2.PyRotationWarper('cylindrical',(K[0,0] + K[1,1]) /2 )

    # initialize the panorama
    # create first mask
    init_mask = np.full(img_array[0].shape[:2],255, dtype="uint8") # white rectangle mask

    # warp first image and mask
    corner, warped = warper.warp(img_array[0], cv2.UMat(K.astype(np.float32)), cv2.UMat(np.eye(3).astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
    _, warped_mask = warper.warp(init_mask, cv2.UMat(K.astype(np.float32)), cv2.UMat(np.eye(3).astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)

    pos_x_init, pos_y_init = corner[0] * -1, corner[1] * -1
    # first image warped and put in 0,0 
    T = np.float32([
                    [1, 0, corner[0] + pos_x_init],
                    [0, 1, corner[1] + pos_y_init]
                    ])
    

    panorama  = cv2.warpAffine(warped.get() , T, (5120, 720))
    warped_mask = cv2.warpAffine(warped_mask.get() , T, (5120, 720))
    known_mask = warped_mask.copy()

    background = cv2.copyMakeBorder(panorama,0,0,0,5280-panorama.shape[1],cv2.BORDER_REPLICATE)
    backSub = cv2.createBackgroundSubtractorKNN()

    backSub.setHistory(20)
    backSub.setDetectShadows(False)

    # initialize the background 
    for i in range(20):
        backSub.apply(background)

    # initialize the prev_R matrix as if there was no rotation
    prev_R = np.eye(3)
    seg_masks = []
    curr_index = 0
    
    tic = time.perf_counter()
    for i in range(1, int(len(img_array)-1) ):
        curr_index = curr_index + img_indexes[i]
        
        if curr_index%100 == 0:
            print(img_indexes[i])
            curr_index=0
            
        # Compute the current update of the angle.
        bg1 = cv2.bitwise_not(bounding_boxes[i])
        bg2 = cv2.bitwise_not(bounding_boxes[i+1])
        update, curr_R = getRotMatrixBetweenImages(img_array[i], img_array[i+1],bg1, bg2) # previous image

        # Use the previous update if the current one cannot be computed.
        if np.isnan(update): 
            print("Nan value...\n", (angle+prev_update)* 180/np.pi)
            update = prev_update 
            curr_R = prev_R
        else:
            prev_update = update
            prev_R = curr_R

        #total rotation update
        tot_R = np.dot(tot_R,curr_R.transpose())
        
        #find homography matrix from rotation
        H = K.dot(tot_R).dot(np.linalg.inv(K))
        H = H / H[2][2]
        angle = angle + update
        angles.append(angle)
        rotation_matricies.append(curr_R)

        #warp image and mask using homography
        # warper.warp outputs corner : x,y of top left corner of warped image
        # warped = warped image 
        corner, warped = warper.warp(img_array[i+1], cv2.UMat(K.astype(np.float32)), cv2.UMat(tot_R.astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
        _, warped_mask = warper.warp(init_mask, cv2.UMat(K.astype(np.float32)), cv2.UMat(tot_R.astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
        
        T = np.float32([
                    [1, 0, corner[0] + pos_x_init],
                    [0, 1, corner[1] + pos_y_init]
                    ])
        
        # warp image and mask and change their size to fit the panorama
        # warp affine displaces the image and puts
        # warped.get() -> get numpy array from warped 

        warped_im = cv2.warpAffine(warped.get() , T, (5120, 720))
        

        warped_mask = cv2.warpAffine(warped_mask.get() , T, (5120, 720))

        # also apply the transformation to ground truth if it exists
        if img_indexes[i] in ground_truth_masks_indexes:

            id = ground_truth_masks_indexes.index(img_indexes[i])
            _,ground_truth_masks[id] = warper.warp(ground_truth_masks[id], cv2.UMat(K.astype(np.float32)), cv2.UMat(np.eye(3).astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
            ground_truth_masks[id] = cv2.warpAffine(ground_truth_masks[id].get() , T, (5120, 720))

      

        # Weighted blending of new image to panorama
        panorama = fuse_panorama_weighted(panorama,warped_im,warped_mask,known_mask, 1)
        known_mask = cv2.bitwise_or(known_mask, warped_mask)

        ## BACKGROUND SUBTRACTION HERE ##
        background = cv2.copyMakeBorder(panorama,0,0,0,5120-panorama.shape[1],cv2.BORDER_REPLICATE)

        # get foreground mask
        fgMask = backSub.apply(background)
        
        """xposition = corner[0] + pos_x_init
        print(xposition)"""
        # crop fgMask to the size of the window
        """start_row = corner[0] + pos_x_init
        end_row = 718 + start_row
        start_col = 0
        end_col = 1126
        fgMask = fgMask[start_row:end_row, start_col:end_col]"""
        _,warped_bb = warper.warp(bounding_boxes[i+1], cv2.UMat(K.astype(np.float32)), cv2.UMat(tot_R.astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
        warped_bb = cv2.warpAffine(warped_bb.get() , T, (5120, 720))
        
        seg_masks.append(masked_image_deletion(fgMask,warped_bb))

        panorama = backSub.getBackgroundImage()
        
    toc = time.perf_counter()
    print(f"Estimated motion masks for sequence in {toc - tic:0.4f} seconds")
    return seg_masks, ground_truth_masks

In [24]:

# select subset of images to run the algorithm
super_step = 4
index_list = list(range(1, len(img_array),4))

for i in ground_truth_masks_indexes:
  if i not in index_list:
    index_list.append(i)

if 500 not in index_list:
  index_list.append(500)
if 501 not in index_list:
  index_list.append(501)
  
index_list.sort()

smaller_img_array = [img_array[i] for i in index_list]

smaller_foreground_masks = [foreground_masks[i] for i in index_list]

seg_masks, ground_truth_masks = getForegroundMasks(smaller_img_array,index_list,ground_truth_masks,ground_truth_masks_indexes,smaller_foreground_masks)

305
409
577
Nan value...
 30.423047793747177
Nan value...
 34.7713143448165
Nan value...
 35.927613195175596
Nan value...
 30.889184589749377
Estimated motion masks for sequence in 162.2189 seconds


In [25]:
def getConfusionMatrixForMask(segMaskTrue, segMaskPred):
    """
    Compute the confusion matrix of a given segmentation mask by
    using the corresponding annotated segmentation mask. The 
    problem is expressed as a binary classification problem 
    where the positive class correspond to the white pixels.

    Parameters
    ----------
    segMaskTrue: A grayscale image that correspond to the annotated 
                  segmentation mask
    segMaskPred: A grayscale image that correspond to the predicted
                  segmentation mask

    Return
    ----------
    confusionMatrix: The computed confusion matrix:
                        ------------------------
                        |Pred\True |  F  |  T  |
                        |----------|-----|-----|
                        |     F    |  TN |  FN |
                        |----------|-----|-----|
                        |     T    |  FP |  TP |
                        ------------------------
    """
    confusionMatrix = np.zeros((2,2))

    for i in range (segMaskTrue.shape[0]):
        for j in range (segMaskTrue.shape[1]):
            if segMaskTrue[i][j] == 0 and segMaskPred[i][j] == 0:
                confusionMatrix[0][0] += 1
            elif segMaskTrue[i][j] == 255 and segMaskPred[i][j] == 0:
                confusionMatrix[0][1] += 1
            elif segMaskTrue[i][j] == 0 and segMaskPred[i][j] == 255:
                confusionMatrix[1][0] += 1
            elif segMaskTrue[i][j] == 255 and segMaskPred[i][j] == 255:
                confusionMatrix[1][1] += 1
    
    return confusionMatrix

## Confusion matrix computation

In [26]:
# use this function to compute blah blah
#confusion_matrix=np.zeros((2,2))

#for i,true_mask_index in enumerate(ground_truth_masks_indexes):
  # get corresponding predicted mask
#  pred_mask = seg_masks[index_list.index(true_mask_index)-1]
#  confusion_matrix += getConfusionMatrixForMask(ground_truth_masks[i],pred_mask)
#print(confusion_matrix)

In [27]:
#print(confusion_matrix/len(ground_truth_masks_indexes))

In [28]:
# save a video of the masks
frameSize = seg_masks[0].shape
out = cv2.VideoWriter('outputcr_hist20_KNN.avi', cv2.VideoWriter_fourcc(*'DIVX'), 6, ((5120,720)),0)
for image in seg_masks:
    out.write(image)
out.release()



In [17]:
cv2.imwrite("KNN_frame_example.png",seg_masks[index_list.index(500)-1])
cv2.imwrite("example_gt.png",ground_truth_masks[ground_truth_masks_indexes.index(1076)])

True

In [ ]:
print(ground_truth_masks[0].shape)
for image in ground_truth_masks:
    if image.shape ==(720,5120):
      #print("yes")
      cv2.imwrite('example_ground_truth.png',image)


## Enhanced panoramic stitching with foreground subtraction

In [ ]:

init_mask = np.full(img_array[first_im].shape[:2],255, dtype="uint8") # white rectangle mask
warper = cv2.PyRotationWarper('cylindrical',(K[0,0] + K[1,1]) /2 )

corner, warped = warper.warp(img_array[first_im], cv2.UMat(K.astype(np.float32)), cv2.UMat(np.eye(3).astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
_, warped_mask = warper.warp(init_mask, cv2.UMat(K.astype(np.float32)), cv2.UMat(np.eye(3).astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
pos_x_init, pos_y_init = corner[0] * -1, corner[1] * -1
T = np.float32([
                [1, 0, corner[0] + pos_x_init],
                [0, 1, corner[1] + pos_y_init]
                ])

panorama  = cv2.warpAffine(warped.get() , T, (5120, 720))
warped_mask = cv2.warpAffine(warped_mask.get() , T, (5120, 720))
known_mask = warped_mask.copy()

panorama_images = []
tic = time.perf_counter()
for i in range(len(rotation_matrices)):

    curr_im_number = first_im + step*i

    # compute homography for current img
    H = K.dot(rotation_matrices[i]).dot(np.linalg.inv(K))
    H = H / H[2][2]

    corner, warped = warper.warp(img_array[curr_im_number+step], cv2.UMat(K.astype(np.float32)), cv2.UMat(rotation_matrices[i].astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
    _, warped_mask = warper.warp(cv2.bitwise_not(foreground_masks[curr_im_number]), cv2.UMat(K.astype(np.float32)), cv2.UMat(rotation_matrices[i].astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)
    
    T = np.float32([
                [1, 0, corner[0] + pos_x_init],
                [0, 1, corner[1] + pos_y_init]
                ])

    warped_im = cv2.warpAffine(warped.get() , T, (5120, 720))
    warped_mask = cv2.warpAffine(warped_mask.get() , T, (5120, 720))

    # use eroded mask to create a highlighting square
    eroder_object = np.ones((10, 10), np.uint8)
    eroded_warped_mask = cv2.erode(	warped_mask, eroder_object)
    highlighter = cv2.bitwise_xor(eroded_warped_mask, warped_mask)

    panorama = fuse_panorama_weighted(panorama,warped_im,warped_mask,known_mask,weight = 0.05)
    
    known_mask = cv2.bitwise_or(known_mask, warped_mask)
    
    highlighted_pano = cv2.bitwise_or(panorama,highlighter)# image with boder higlight to make video mater
    
    panorama_images.append(highlighted_pano)

toc = time.perf_counter()
print(f" Stitched sequence in {toc - tic:0.4f} seconds")

cv2.imwrite("pano final.jpg",panorama)    
cv2.imshow("final pano", panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()

 Stitched sequence in 11.7489 seconds


QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to target thread (0x55dc6586ab40)

QObject::moveToThread: Current thread (0x55dc6586ab40) is not the object's thread (0x55dd11aec120).
Cannot move to tar

## Video of panorama stitching process

In [ ]:
out = cv2.VideoWriter('pano_no_back_test_seq.avi',cv2.VideoWriter_fourcc(*'DIVX'), 12, (5120,720), 0)
for i, img in enumerate(panorama_images):
    img = writeAngle(img,round(angles[i] * 180/np.pi, 2))
    out.write(img)

out.release()

## Figure Generation

In [ ]:
sift = cv2.SIFT.create(nfeatures = 2000)
im_no = 500
kp1, des1 = sift.detectAndCompute(img_array[im_no],cv2.bitwise_not(foreground_masks[im_no]))

im=cv2.drawKeypoints(img_array[im_no],kp1,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imwrite("sift_point.jpg",im)

cv2.imshow("sift", im)

cv2.waitKey(0)
cv2.destroyAllWindows()


warper = cv2.PyRotationWarper('cylindrical',(K[0,0] + K[1,1]) /2 )
corner, warped = warper.warp(img_array[im_no], cv2.UMat(K.astype(np.float32)), cv2.UMat(np.eye(3).astype(np.float32)),cv2.INTER_CUBIC , cv2.BORDER_CONSTANT)

cv2.imwrite("warped_im.jpg",warped)
cv2.imshow("warped", warped)

cv2.waitKey(0)
cv2.destroyAllWindows()

mask = np.full((size[1],size[0]),0, dtype="uint8") #black mask
cv2.rectangle(mask, [1177.3617,  201.9200, 1278.5439,  356.1601], 255, -1) # white rectangles
cv2.rectangle(mask, [ 970.7972,   90.5755, 1173.3964,  391.2020], 255, -1) # white rectangles
cv2.rectangle(mask, [ 85.2200,    83.0779,  246.5678,  501.0777], 255, -1) # white rectangles
cv2.rectangle(mask, [ 936.6094,  112.2774, 1000.6503,  179.6730], 255, -1) # white rectangles